In [1]:
import torch
from torch.optim import Adam
import dset, model # by importing dset, we also import the text file and the related data.
import numpy as np
import datetime
import utils
char_size = dset.char_size
from torch.utils.data import DataLoader # batch

In [2]:
MODEL_STR = "LSTM"
train_size = 25 # number of characters used for single iteration (single computation of loss)
h_size = 512 # size of hidden state
depth = 1 # number of hidden layers in RNN
batch_size = 512
device = 'cuda'#torch.device("cuda" if torch.cuda.is_available() else torch.device("cpu"))
print(f"training on {device}.")

training on cuda.


In [3]:
rnn_model = getattr(model,MODEL_STR)
rnn = rnn_model(char_size, 
                char_size, 
                h_size,
                depth,
                dropout = 0.5
               ).to(device) # define model

In [4]:
# gradient clipping, https://stackoverflow.com/questions/54716377/how-to-do-gradient-clipping-in-pytorch
clip_value=1.
for p in rnn.parameters():
    p.register_hook(lambda grad: torch.clamp(grad, -clip_value, clip_value))

optimizer = Adam(rnn.parameters(),lr=0.01) # 0.001 for RNN, 0.01 for LSTM and GRU work reasonably

max_iter = 5000 # maximum number of training iterations
current_iter = 0 # counter for training iterations

# print the loss 11 times during the course of training
print_idx_list = [1]+[int(np.floor(i*max_iter/10)) for i in range(1,11)] 

loop_bool = True

if MODEL_STR == "LSTM":
    h = ([torch.zeros(1, h_size, device=device) for i in range(depth)],[torch.zeros(1, h_size, device=device) for i in range(depth)])
elif MODEL_STR == "GRU" or MODEL_STR == "RNN":
    h = [torch.zeros(1, h_size, device=device) for i in range(depth)]
else:
    raise NotImplementedError

data = dset.textDataset(dset.text, train_size) # data: iterable with text given in chunks of size train_size (size counted by number of characters)
train_dl = DataLoader(
    data,
    batch_size=batch_size,
    pin_memory=torch.cuda.is_available()
)

rnn.train()
t1 = datetime.datetime.now()
while loop_bool:
    for batch_txt in train_dl:
        if current_iter<max_iter:
            current_iter+=1
            optimizer.zero_grad()
            loss=utils.compute_loss(rnn,batch_txt,h,device)
            loss.backward()
            optimizer.step()
            if current_iter in print_idx_list:
                print('Iter: {} out of {}'.format(current_iter,max_iter),'// loss: ',loss.detach())
        else:
            loop_bool = False
            break
t2 = datetime.datetime.now()
print("Training time:\n",t2-t1)

Iter: 1 out of 5000 // loss:  tensor(112.4639, device='cuda:0')
Iter: 500 out of 5000 // loss:  tensor(37.8586, device='cuda:0')
Iter: 1000 out of 5000 // loss:  tensor(32.1557, device='cuda:0')
Iter: 1500 out of 5000 // loss:  tensor(30.9986, device='cuda:0')
Iter: 2000 out of 5000 // loss:  tensor(29.2055, device='cuda:0')
Iter: 2500 out of 5000 // loss:  tensor(28.2020, device='cuda:0')
Iter: 3000 out of 5000 // loss:  tensor(28.2304, device='cuda:0')
Iter: 3500 out of 5000 // loss:  tensor(27.6190, device='cuda:0')
Iter: 4000 out of 5000 // loss:  tensor(26.8322, device='cuda:0')
Iter: 4500 out of 5000 // loss:  tensor(26.5563, device='cuda:0')
Iter: 5000 out of 5000 // loss:  tensor(26.3475, device='cuda:0')
Training time:
 0:47:01.527142


In [5]:
utils.print_function(2000,rnn.to('cpu'),char_size,h_size,depth,MODEL_STR)

ingley had any consequence
      away; but before he realons of the London; and you know he is appearance in the most ation as he
      stood and make her three great
      interest.”

      “My dear Mr. Darcy, for she had not the messen, on better.”

      “And but it was to deter him frequenting for
      any comparison pourly a cross of his quiet readily anxiety to see you for it.”

      “He is not a deep it is planess and a
      dispostning, gentlemen provesed and
      behaviour of nothing
      encourageness than between the world I shall do you den sure in he was
      contradied, and that they made her expected their
      _tête-à-tête_ a be the
      steady my dear beyond young our wishes, nor was Mr. Bingley and I will do the obliging what they achonsed.

      Mr. Darcy believed, you know the whole windownce out as much and I have been proposals; for what I thought as before, seemed to listen a lively, for a lice him
      only in Hertfordshire a gentlemen and the former c